In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.dirname("RECSYS_POLIMI_2024"), '..'))


from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

URM_all_dataframe = pd.read_csv(filepath_or_buffer="data_train.csv")
ICM_all = pd.read_csv(filepath_or_buffer="data_ICM_metadata.csv")
URM_all_dataframe.head(n=10)


k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]

URM_train_validation, URM_test = train_test_split(URM_all_dataframe, test_size=0.2, random_state=42)
URM_train, URM_validation = train_test_split(URM_train_validation, test_size=0.2, random_state=42)

# # Reset indices for cleaner output (optional)
# URM_test = URM_test.reset_index(drop=True)
# URM_validation = URM_validation.reset_index(drop=True)
# URM_train = URM_train.reset_index(drop=True)
# URM_train_validation = URM_train_validation.reset_index(drop=True)

# Display the results
print("URM_train (80% of URM_ALL):")
print(URM_train.head(n=10))

print("\nURM_validation (20% of URM_ALL):")
print(URM_validation.head(n=10))

print("\nURM_test (20% of URM_ALL):")
print(URM_test.head(n=10))


URM_train (80% of URM_ALL):
         user_id  item_id  data
136935      1523    11379   1.0
286638      3415    29647   1.0
499343      6571    11101   1.0
1612162    31036    17962   1.0
1231767    22131    20291   1.0
1419045    25670    32878   1.0
505970      6677    30720   1.0
1309905    23612    35978   1.0
1026162    17411     9221   1.0
991787     16720     2668   1.0

URM_validation (20% of URM_ALL):
         user_id  item_id  data
1293463    23345    37683   1.0
978469     16471     1559   1.0
1139447    19988     6956   1.0
628101      8833      614   1.0
1276102    23100    20305   1.0
570596      7748      117   1.0
1232901    22148    35968   1.0
1544801    29212    33031   1.0
1033303    17563    34624   1.0
658881      9451     9732   1.0

URM_test (20% of URM_ALL):
         user_id  item_id  data
1092502    18939     8673   1.0
367273      4669     2436   1.0
1181580    20972    25295   1.0
1243868    22394    23927   1.0
302903      3680    24294   1.0
1337700    240

In [2]:
URM_all_dataframe = csr_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_train = csr_matrix((URM_train['data'], (URM_train['user_id'], URM_train['item_id'])))
URM_validation = csr_matrix((URM_validation['data'], (URM_validation['user_id'], URM_validation['item_id'])))
URM_test = csr_matrix((URM_test['data'], (URM_test['user_id'], URM_test['item_id'])))
URM_train_validation = csr_matrix((URM_train_validation['data'], (URM_train_validation['user_id'], URM_train_validation['item_id'])))
ICM_all = csr_matrix((ICM_all['data'], (ICM_all['item_id'], ICM_all['feature_id'])))

#evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 401 ( 1.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 152 ( 0.4%) Users that have less than 1 test interactions


## Final hybrid

In [44]:
from Recommenders.SLIM.MSLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_all_dataframe, ICM_train=None, weigth=1.0 )

slim.fit( 
    l1_ratio=0.49705634165029133,
   alpha=9.773610229057873e-05,
   topK=185
                   )

Progress: 100%|██████████| 38121/38121 [09:51<00:00, 64.41items/s] 


In [45]:
slim.save_model(folder_path="result_experiments", file_name="SLIM_BEST_MODEL_ALL_DATA")

SLIMElasticNetRecommender: Saving model in file 'result_experimentsSLIM_BEST_MODEL_ALL_DATA'
SLIMElasticNetRecommender: Saving complete


In [3]:
from Recommenders.SLIM.MSLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
slim = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_all_dataframe, ICM_train=None, weigth=1.0 )
slim.load_model(folder_path="result_experiments", file_name="SLIM_BEST_MODEL_ALL_DATA")

SLIMElasticNetRecommender: Loading model from file 'result_experimentsSLIM_BEST_MODEL_ALL_DATA'
SLIMElasticNetRecommender: Loading complete


In [4]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
rp3 = RP3betaRecommender(URM_all_dataframe)
rp3.fit( 
    topK=15,
    alpha=0.32714863414672785,
    beta=0.23057896475734277,
    normalize_similarity=True
                    )

RP3betaRecommender: Similarity column 38121 (100.0%), 4023.90 column/sec. Elapsed time 9.47 sec


In [11]:
import sys
import logging

import torch
import pandas as pd


class TorchEASE:
    def __init__(
        self, train, user_col="user_id", item_col="item_id", score_col=None, reg=250.0
    ):
        """

        :param train: Training DataFrame of user, item, score(optional) values
        :param user_col: Column name for users
        :param item_col: Column name for items
        :param score_col: Column name for scores. Implicit feedback otherwise
        :param reg: Regularization parameter.
                    Change by orders of magnitude to tune (2e1, 2e2, ...,2e4)
        """
        logging.basicConfig(
            format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
            level=logging.INFO,
            datefmt="%Y-%m-%d %H:%M:%S",
            stream=sys.stdout,
        )

        self.logger = logging.getLogger("notebook")
        self.logger.info("Building user + item lookup")
        # How much regularization do you need?
        self.reg = reg

        self.user_col = user_col
        self.item_col = item_col

        self.user_id_col = user_col + "_id"
        self.item_id_col = item_col + "_id"

        self.user_lookup = self.generate_labels(train, self.user_col)
        self.item_lookup = self.generate_labels(train, self.item_col)

        self.item_map = {}
        self.logger.info("Building item hashmap")
        for _item, _item_id in self.item_lookup.values:
            self.item_map[_item_id] = _item

        train = pd.merge(train, self.user_lookup, on=[self.user_col])
        train = pd.merge(train, self.item_lookup, on=[self.item_col])
        self.logger.info("User + item lookup complete")
        self.indices = torch.LongTensor(
            train[[self.user_id_col, self.item_id_col]].values
        )

        if not score_col:
            # Implicit values only
            self.values = torch.ones(self.indices.shape[0])
        else:
            self.values = torch.FloatTensor(train[score_col])

        # TODO: Is Sparse the best implementation?
        self.sparse = torch.sparse.FloatTensor(self.indices.t(), self.values)

        self.logger.info("Sparse data built")

    def generate_labels(self, df, col):
        dist_labels = df[[col]].drop_duplicates()
        dist_labels[col + "_id"] = dist_labels[col].astype("category").cat.codes

        return dist_labels

    def fit(self):
        self.logger.info("Building G Matrix")
        print(f"sto fittando con reg = {self.reg} ")
        G = self.sparse.to_dense().t() @ self.sparse.to_dense()
        G += torch.eye(G.shape[0]) * self.reg

        P = G.inverse()

        self.logger.info("Building B matrix")
        B = P / (-1 * P.diag())
        # Set diagonals to 0. TODO: Use .fill_diag_
        B = B + torch.eye(B.shape[0])

        # Predictions for user `_u` will be self.sparse.to_dense()[_u]@self.B
        self.B = B

        return

    def predict_all(self, pred_df, k=5, remove_owned=True):
        """
        :param pred_df: DataFrame of users that need predictions
        :param k: Number of items to recommend to each user
        :param remove_owned: Do you want previously interacted items included?
        :return: DataFrame of users + their predictions in sorted order
        """
        pred_df = pred_df[[self.user_col]].drop_duplicates()
        n_orig = pred_df.shape[0]

        # Alert to number of dropped users in prediction set
        pred_df = pd.merge(pred_df, self.user_lookup, on=[self.user_col])
        n_curr = pred_df.shape[0]
        if n_orig - n_curr:
            self.logger.info(
                "Number of unknown users from prediction data = %i" % (n_orig - n_curr)
            )

        _output_preds = []
        # Select only user_ids in our user data
        _user_tensor = self.sparse.to_dense().index_select(
            dim=0, index=torch.LongTensor(pred_df[self.user_id_col])
        )

        # Make our (raw) predictions
        _preds_tensor = _user_tensor @ self.B
        self.logger.info("Predictions are made")
        if remove_owned:
            # Discount these items by a large factor (much faster than list comp.)
            self.logger.info("Removing owned items")
            _preds_tensor += -1.0 * _user_tensor

        self.logger.info("TopK selected per user")
        for _preds in _preds_tensor:
            # Very quick to use .topk() vs. argmax()
            _output_preds.append(
                [self.item_map[_id] for _id in _preds.topk(k).indices.tolist()]
            )

        pred_df["predicted_items"] = _output_preds
        self.logger.info("Predictions are returned to user")
        return pred_df

    def score_predictions(self, pred_df, true_interactions, k=5):
        """
        :param pred_df: DataFrame containing predicted items for each user
        :param true_interactions: DataFrame with true user-item interactions (user_id, item_id)
        :param k: Number of top recommendations to consider
        :return: Dictionary with Precision@K and Recall@K
        """

        # Merge true interactions with the prediction DataFrame
        pred_df = pred_df.rename(columns={"predicted_items": "predicted_items"})

        # Initialize metrics
        precision_at_k = 0
        recall_at_k = 0
        total_users = len(pred_df)

        # Iterate over each user in the prediction DataFrame
        for _, row in pred_df.iterrows():
            user_id = row[self.user_col]
            recommended_items = set(row["predicted_items"])

            # Get true interacted items for this user
            true_items = set(true_interactions[true_interactions[self.user_col] == user_id][self.item_col])

            # Calculate precision and recall for this user
            common_items = recommended_items.intersection(true_items)

            # Precision: the fraction of relevant items in top-k recommendations
            precision_at_k += len(common_items) / k

            # Recall: the fraction of relevant items retrieved in top-k recommendations
            recall_at_k += len(common_items) / len(true_items) if true_items else 0

        # Compute average precision and recall over all users
        precision_at_k /= total_users
        recall_at_k /= total_users

        return {"precision_at_k": precision_at_k, "recall_at_k": recall_at_k}

    def get_URM_train(self):
        """
        Returns the training User-Item Rating Matrix in SciPy CSR format.
        Required by EvaluatorHoldout.
        """
        return self.to_scipy()

    def recommend(self, user_ids, remove_seen_flag=True, cutoff=10, **kwargs):
        """
        Generates recommendations for a batch of users.
        Compatible with EvaluatorHoldout.

        :param user_ids: Array of user IDs (internal IDs as used in the matrix)
        :param remove_seen_flag: Whether to exclude already interacted items
        :param cutoff: Number of recommendations to return per user
        :return: List of recommended items for each user
        """
        # Convert user_ids to tensor for indexing
        user_tensor = torch.LongTensor(user_ids)

        # Compute raw scores using the B matrix
        user_scores = self.sparse.to_dense().index_select(dim=0, index=user_tensor) @ self.B

        if remove_seen_flag:
            # Remove previously seen items by penalizing their scores
            user_interactions = self.sparse.to_dense().index_select(dim=0, index=user_tensor)
            user_scores += -1.0 * user_interactions

        recommended_items = []
        for scores_tensor in user_scores:
            top_indices = scores_tensor.topk(cutoff).indices
            recommended_items.append(top_indices.tolist())

        return recommended_items, None  # Return second value as `None` for compatibility

    def prepare_users(self, user_list):
        """
        Prepares a list of user IDs for compatibility with the evaluation function.

        :param user_list: List of external user IDs.
        :return: List of internal user IDs corresponding to the input.
        """
        return self.user_lookup[self.user_lookup[self.user_col].isin(user_list)][self.user_id_col].tolist()

In [31]:
# Crea l'istanza del modello
urm = pd.read_csv('data_train.csv')
urm_train_validation, urm_test = train_test_split(urm, test_size=0.2, random_state=42)
ease = TorchEASE(urm, user_col='user_id', item_col='item_id', reg=50)

# Allena il modello
ease.fit()


2024-12-28 11:03:38 [INFO] notebook - Building user + item lookup
2024-12-28 11:03:38 [INFO] notebook - Building item hashmap
2024-12-28 11:03:38 [INFO] notebook - User + item lookup complete
2024-12-28 11:03:38 [INFO] notebook - Sparse data built
2024-12-28 11:03:38 [INFO] notebook - Building G Matrix
sto fittando con reg = 50 
2024-12-28 11:09:32 [INFO] notebook - Building B matrix


In [50]:
from Recommenders.EASE_R.EASE_TORCH import EASE_R_Recommender

ease = EASE_R_Recommender(URM_all_dataframe)
ease.load_model(folder_path="result_experiments", file_name="EASE_REG50_ALL_DATA")

2024-12-28 20:32:43 [INFO] notebook - Building user + item lookup
2024-12-28 20:32:43 [INFO] notebook - Converting sparse matrix to DataFrame for lookup generation
2024-12-28 20:32:43 [INFO] notebook - Building item hashmap
2024-12-28 20:32:43 [INFO] notebook - User + item lookup complete
2024-12-28 20:32:43 [INFO] notebook - Sparse data built
EASE_R_Recommender: Loading model from file 'result_experimentsEASE_REG50_ALL_DATA'
EASE_R_Recommender: Loading complete
EASE_R_Recommender: B matrix not found during loading. Ensure it was saved correctly.


In [ ]:
W = csr_matrix(ease.B)

In [17]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender


ease_r = ItemKNNCustomSimilarityRecommender(URM_train_validation)
ease_r.fit(W)

In [5]:
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
ease_r = ItemKNNCustomSimilarityRecommender(URM_all_dataframe)
ease_r.load_model(folder_path="result_experiments", file_name="EASE_REG50_ALL_DATA")

ItemKNNCustomSimilarityRecommender: Loading model from file 'result_experimentsEASE_REG50_ALL_DATA'
ItemKNNCustomSimilarityRecommender: Loading complete


In [ ]:
from Recommenders.MatrixFactorization.IALSJnoearly import ImplicitALSRecommender
als = ImplicitALSRecommender(URM_all_dataframe)
als.fit( epochs=6,
                    num_factors=1650,
                    confidence_scaling="linear",
                    alpha=6,
                    epsilon=1.0,
                    reg=1e-4,
                    use_gpu=False,
                    num_threads=8,
                    calculate_training_loss=True
                    )

In [11]:
import scipy.sparse as sps
from Recommenders.BaseRecommender import BaseRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

class W4HybridRecommender( ItemKNNCustomSimilarityRecommender ):

    def __init__(self, URM_train, rec1, rec2, rec3, rec4):
        self.URM_train = sps.csr_matrix(URM_train)
        super(W4HybridRecommender, self).__init__(URM_train)
        self.rec=None
        self.recommend1=rec1
        self.recommend2=rec2
        self.recommend3=rec3
        self.recommend4=rec4
        self.gamma = None

    def fit(self, alpha = 0.5, beta = 0.5, gamma = 0.5):
        self.alpha = alpha
        self.gamma = gamma
        new_similarity = (
        alpha * csr_matrix(self.recommend1) +
        beta * csr_matrix(self.recommend2) +
        (1 - alpha - beta) * csr_matrix(self.recommend3)
    )
        recommender_object = ItemKNNCustomSimilarityRecommender(URM_all_dataframe)
        recommender_object.fit(new_similarity)
        self.rec=recommender_object

    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.rec._compute_item_score(user_id_array)
        item_weights_2 = self.recommend4._compute_item_score(user_id_array)
        

        item_weights = item_weights_1*self.gamma + item_weights_2*(1-self.gamma)

        return item_weights

In [ ]:
slimrp3easeals = W4HybridRecommender(URM_all_dataframe, slim.W_sparse, rp3.W_sparse, ease_r.W_sparse, als)
slimrp3easeals.fit( 
                    alpha = 0.6153342470415742,
                    beta = 0.24391799534575526,
                    gamma = 0.8179934109860636
                )

In [64]:
result_df, _ = evaluator_test.evaluateRecommender(slimrp3easeals)
result_df

EvaluatorHoldout: Processed 35584 (100.0%) in 1.23 min. Users per second: 483


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.995747,0.0,0.995747,0.253168,13.558472,0.999904,0.414094,0.918239,1.235463,0.196302


In [66]:
recommendations_list = []
user = pd.read_csv(filepath_or_buffer="data_target_users_test.csv")

# Loop through each user and get the top 10 recommendations
for user_id in user["user_id"]:  # assuming URM.shape[0] is the total number of users
    top_10_recommendations = slimrp3easeals.recommend(user_id, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=True)
    # Join recommendations into a space-separated string
    item_list_str = ' '.join(map(str, top_10_recommendations))
    recommendations_list.append([user_id, item_list_str])
# Convert the list to a DataFrame
recommendations_df = pd.DataFrame(recommendations_list, columns=['user_id', 'item_list'])

# Display the resulting table
print(recommendations_df.head(2))

# Optionally, save to CSV
recommendations_df.to_csv('top_10_recommendationsHybridSLIMRP3easealscorrect.csv', index=False)

   user_id                                          item_list
0        0  7547 572 9911 7703 14888 1425 14931 3074 3146 531
1        1  6348 14748 13733 13766 572 11875 9911 8685 264...


## Tuning of final hybrid

In [40]:
import optuna
import pandas as pd
def objective_function(optuna_trial):
    
    recommender_instance = W4HybridRecommender(URM_train_validation, slim.W_sparse, rp3.W_sparse, W, als)
    alpha = optuna_trial.suggest_uniform('alpha', 0.46, 0.68)
    
    # Use alpha to set the range for beta
    beta = optuna_trial.suggest_uniform('beta', 0.2, 0.3)
    
    # Define gamma
    gamma = optuna_trial.suggest_uniform('gamma', 0.8, 1)
    
    recommender_instance.fit(
        alpha=alpha,
        beta=beta,
        gamma=gamma
    )
    
    result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
    return result_df.loc[10]["MAP"]

In [36]:
class SaveResults(object):
    def __init__(self, save_path="SLIMRP3EASE_Rals.csv"):
        # File dove salvare i risultati
        self.save_path = save_path

        # Se esiste già un file, carichiamo i dati per continuare
        try:
            self.results_df = pd.read_csv(self.save_path)
        except FileNotFoundError:
            self.results_df = pd.DataFrame(columns=["trial_number", "value", "params"])
    
    def __call__(self, optuna_study, optuna_trial):
        # Salva i risultati del trial corrente
        trial_data = {
            "trial_number": optuna_trial.number,
            "value": optuna_trial.value,
            "params": optuna_trial.params
        }

        # Aggiungi i risultati al DataFrame
        self.results_df = pd.concat([self.results_df, pd.DataFrame([trial_data])], ignore_index=True)

        # Salva i risultati progressivamente su disco
        self.results_df.to_csv(self.save_path, index=False)

        # Salva anche il miglior risultato corrente
        best_trial = optuna_study.best_trial
        best_result = {
            "trial_number": best_trial.number,
            "value": best_trial.value,
            **best_trial.params
        }

        best_results_df = pd.DataFrame([best_result])
        best_results_df.to_csv("bestSLIMRP3EASE_Rals.csv", index=False)
        
        print(f"Results saved to {self.save_path} and best results to 'best_optuna_results.csv'")


In [41]:
optuna_study = optuna.create_study(direction="maximize")

# Inizializzazione del callback per salvare progressivamente i risultati
save_results = SaveResults(save_path="optuna_progressSLIMRP3EASE_Rals2.csv")

# Esecuzione dell'ottimizzazione
optuna_study.optimize(
    objective_function,
    callbacks=[save_results],
    n_trials=400
)

[I 2024-12-28 16:13:39,018] A new study created in memory with name: no-name-d4bc766f-af89-499a-aed1-43edeb5372d4
C:\Users\vgius\AppData\Local\Temp\ipykernel_37392\2136426073.py:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  alpha = optuna_trial.suggest_uniform('alpha', 0.46, 0.68)
C:\Users\vgius\AppData\Local\Temp\ipykernel_37392\2136426073.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  beta = optuna_trial.suggest_uniform('beta', 0.2, 0.3)
C:\Users\vgius\AppData\Local\Temp\ipykernel_37392\2136426073.py:12: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gamma

EvaluatorHoldout: Processed 35584 (100.0%) in 1.07 min. Users per second: 556


[I 2024-12-28 16:14:48,612] Trial 0 finished with value: 0.062225612367247066 and parameters: {'alpha': 0.5261342398864439, 'beta': 0.2359925368108785, 'gamma': 0.9219340946815249}. Best is trial 0 with value: 0.062225612367247066.


Results saved to optuna_progressSLIMRP3EASE_Rals2.csv and best results to 'best_optuna_results.csv'
EvaluatorHoldout: Processed 35584 (100.0%) in 1.09 min. Users per second: 543


[I 2024-12-28 16:15:59,598] Trial 1 finished with value: 0.06211935250906203 and parameters: {'alpha': 0.5543794736290666, 'beta': 0.2281894561716052, 'gamma': 0.944363823094278}. Best is trial 0 with value: 0.062225612367247066.


...

Results saved to optuna_progressSLIMRP3EASE_Rals2.csv and best results to 'best_optuna_results.csv'
EvaluatorHoldout: Processed 35584 (100.0%) in 1.11 min. Users per second: 535


[I 2024-12-28 20:09:35,183] Trial 200 finished with value: 0.06233974316082616 and parameters: {'alpha': 0.6249479115910231, 'beta': 0.24602611595502732, 'gamma': 0.8196185554948173}. Best is trial 124 with value: 0.06236474659557851.


Results saved to optuna_progressSLIMRP3EASE_Rals2.csv and best results to 'best_optuna_results.csv'


[W 2024-12-28 20:10:09,107] Trial 201 failed with parameters: {'alpha': 0.6155551291039123, 'beta': 0.2550770235205882, 'gamma': 0.8149434960955612} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\vgius\anaconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\vgius\AppData\Local\Temp\ipykernel_37392\2136426073.py", line 20, in objective_function
    result_df, _ = evaluator_test.evaluateRecommender(recommender_instance)
  File "c:\Users\vgius\Documents\GitHub\RECSYS_POLIMI_2024\1OUR_NOTEBOOKS\..\Evaluation\Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "c:\Users\vgius\Documents\GitHub\RECSYS_POLIMI_2024\1OUR_NOTEBOOKS\..\Evaluation\Evaluator.py", line 476, in _run_evaluation_on_selected_users
    recommended_items_batch_list, sc

KeyboardInterrupt: 

In [42]:
import pandas as pd

# Leggi il file CSV
csv_file = "optuna_progressSLIMRP3EASE_Rals2.csv"  # Sostituisci con il nome effettivo del file
df = pd.read_csv(csv_file)

# Ordina il DataFrame in base alla colonna 'value' in ordine decrescente
df_sorted = df.sort_values(by="value", ascending=False)

# Mostra il DataFrame ordinato (opzionale)
display(df_sorted)

# Salva il DataFrame ordinato in un nuovo file CSV
sorted_csv_file = "nome_del_file_ordinato4.csv"  # Sostituisci con il nome desiderato
df_sorted.to_csv(sorted_csv_file, index=False)

print(f"File ordinato salvato come {sorted_csv_file}")


,trial_number,value,params
124,124,0.062365,"{'alpha': 0.6153342470415742, 'beta': 0.243917..."
176,176,0.062361,"{'alpha': 0.6177253394844043, 'beta': 0.245505..."
122,122,0.062361,"{'alpha': 0.6156564719491163, 'beta': 0.244421..."
110,110,0.062360,"{'alpha': 0.6108025334472509, 'beta': 0.244691..."
137,137,0.062355,"{'alpha': 0.6039889642591557, 'beta': 0.242712..."
...,...,...,...
7,7,0.061891,"{'alpha': 0.5090686683781133, 'beta': 0.252730..."
5,5,0.061889,"{'alpha': 0.6285915942269922, 'beta': 0.276335..."
116,116,0.061848,"{'alpha': 0.6186800711035859, 'beta': 0.251945..."
50,50,0.061801,"{'alpha': 0.5838800167035303, 'beta': 0.273875..."


File ordinato salvato come nome_del_file_ordinato4.csv
